In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install transformers torch opencv-python Pillow tqdm


In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm
import numpy as np
from PIL import Image

def extract_frames(video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    # Get video properties
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video Info: {frame_count} frames, {fps} FPS, {width}x{height} resolution.")
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Add frame if the index is a multiple of frame_interval
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

def generate_video_caption(video_path, frame_interval=30):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, frame_interval)
    
    if not frames:
        return []  # No frames to process if extraction failed
    
    captions = []
    
    for i, frame in tqdm(enumerate(frames), total=len(frames), desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
        print(f"Generated caption for frame {i+1}: {caption}")
    
    return captions

def save_captions(captions, output_file="captions.txt"):
    with open(output_file, "w") as f:
        for idx, caption in enumerate(captions):
            f.write(f"Frame {idx+1}: {caption}\n")

# Example usage
video_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/.cartoonmovies7_7298084692916555041.mp4"  # Replace with your video file path
captions = generate_video_caption(video_path)

# Save captions to a file if captions are not empty
if captions:
    save_captions(captions)

# Print the first few captions if they exist
if captions:
    for i in range(min(5, len(captions))):
        print(f"Frame {i+1}: {captions[i]}")
else:
    print("No captions generated.")


In [ ]:
import os
import cv2
import torch
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm
from PIL import Image

def extract_frames(video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Add frame if the index is a multiple of frame_interval
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    return frames

def generate_video_caption(video_path, frame_interval=30):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, frame_interval)
    
    if not frames:
        return ""  # Return empty string if no frames were processed
    
    captions = []
    for frame in tqdm(frames, desc=f"Generating captions for {os.path.basename(video_path)}"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    
    # Concatenate captions for all frames of the video
    return " ".join(captions)

def process_videos_in_directory(input_dir, output_dir, frame_interval=30):
    # Ensure the output directory exists
    os.makedirs(output_dir, exist_ok=True)
    
    # Path to the output CSV file
    output_csv = os.path.join(output_dir, "video_captions.csv")
    
    # Initialize a DataFrame to store video names and captions
    video_captions = []

    # Iterate over all video files in the directory
    for video_file in os.listdir(input_dir):
        if video_file.endswith((".mp4", ".avi", ".mkv")):  # Check for video extensions
            video_path = os.path.join(input_dir, video_file)
            print(f"Processing video: {video_file}")
            
            # Generate captions for the video
            captions = generate_video_caption(video_path, frame_interval)
            
            # Add to the DataFrame
            video_captions.append({"video_name": video_file, "caption": captions})

    # Save the results to a CSV file
    df = pd.DataFrame(video_captions)
    df.to_csv(output_csv, index=False)
    print(f"Captions saved to {output_csv}")

# Example usage in Kaggle
input_directory = "/kaggle/input/tikharm-dataset/Dataset/train/Safe"  # Replace with your Kaggle input dataset folder
output_directory = "/kaggle/working"  # Default writable directory in Kaggle
frame_interval = 30  # Adjust as needed

process_videos_in_directory(input_directory, output_directory, frame_interval)


In [ ]:
!pip install transformers torch opencv-python Pillow tqdm moviepy SpeechRecognition


In [ ]:
!pip install moviepy

In [ ]:
pip install opencv-python transformers pillow tqdm pydub speechrecognition


In [ ]:
pip install opencv-python transformers pillow tqdm pydub speechrecognition


In [ ]:
import cv2
import torch
from transformers import BlipProcessor, BlipForConditionalGeneration
from tqdm import tqdm
import numpy as np
from PIL import Image
import os
import pandas as pd
from moviepy.editor import VideoFileClip
import speech_recognition as sr

def extract_frames(video_path, frame_interval=30):
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    fps = cap.get(cv2.CAP_PROP_FPS)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    
    print(f"Video Info: {frame_count} frames, {fps} FPS, {width}x{height} resolution.")
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

def generate_video_caption(video_path, frame_interval=30):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, frame_interval)
    
    if not frames:
        return ""  
    
    captions = []
    
    for i, frame in tqdm(enumerate(frames), total=len(frames), desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
        print(f"Generated caption for frame {i+1}: {caption}")
    
    full_caption = " ".join(captions)
    return full_caption

def extract_audio(video_path):
    """Extract audio from video."""
    video = VideoFileClip(video_path)
    audio = video.audio
    return audio

def transcribe_audio(audio):
    """Transcribe audio to text using SpeechRecognition."""
    recognizer = sr.Recognizer()
    with sr.AudioData(audio.to_soundarray(fps=16000), 16000, 2) as source:
        audio_data = recognizer.record(source)
    
    try:
        transcription = recognizer.recognize_google(audio_data)
        return transcription
    except sr.UnknownValueError:
        return "Audio could not be understood"
    except sr.RequestError as e:
        return f"Error with the request: {e}"

def process_single_video(video_path, output_file="single_video_caption.csv", frame_interval=30):
    # Generate captions from video frames
    full_caption = generate_video_caption(video_path, frame_interval)

    if full_caption:
        # Extract and transcribe the audio directly without saving it as a file
        audio = extract_audio(video_path)
        audio_transcription = transcribe_audio(audio)

        # Prepare data to be saved
        video_data = [{
            "video_name": os.path.basename(video_path),
            "caption": full_caption,
            "audio_transcription": audio_transcription
        }]
        
        # Save data to CSV
        df = pd.DataFrame(video_data)
        df.to_csv(output_file, index=False)
        print(f"Captions and audio transcriptions saved to {output_file}")
    else:
        print("No captions generated for this video.")

# Example usage for a single video
video_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/.cartoonmovies7_7298084692916555041.mp4"  # Replace with the path to your video
output_file = "/kaggle/working/single_video_caption.csv"  # Output CSV file path

process_single_video(video_path, output_file)


In [ ]:
import cv2
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm
import tempfile

# Function to extract frames from video
def extract_frames(video_path, frame_interval=180):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Add frame if the index is a multiple of frame_interval
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

# Function to generate video captions from frames
def generate_video_caption(video_path, frame_interval=180):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, frame_interval)
    
    if not frames:
        return ""  # No frames to process if extraction failed
    
    captions = []
    
    for frame in tqdm(frames, desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    
    return " ".join(captions)  # Concatenate all captions into one string

# Function to extract audio and convert to text using speech recognition
def extract_audio_text(video_path):
    # Use ffmpeg to extract audio
    audio_path = "temp_audio.wav"
    os.system(f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 2 {audio_path}")
    
    # Use speech recognition to transcribe the audio
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)  # read the entire audio file
        try:
            transcript = recognizer.recognize_google(audio)
            os.remove(audio_path)  # Delete the temp audio file
            return transcript
        except sr.UnknownValueError:
            return "Audio could not be transcribed"
        except sr.RequestError as e:
            return f"Could not request results from Google Speech Recognition service; {e}"

# Main function to process the video and save captions/audio transcript to CSV
def process_video(video_path, output_csv="video_captions.csv"):
    caption = generate_video_caption(video_path)
    audio_transcript = extract_audio_text(video_path)
    
    data = {
        "video_name": os.path.basename(video_path),
        "caption": caption,
        "audio_transcript": audio_transcript
    }

    df = pd.DataFrame([data])
    
    # Append to CSV file
    df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

# Example usage
video_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/.cartoonmovies7_7298084692916555041.mp4"  # Replace with your video file path
process_video(video_path)  # This will process the video and save captions and audio text to CSV


In [ ]:
import cv2
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm

# Function to extract frames from video
def extract_frames(video_path, frame_interval=180):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Add frame if the index is a multiple of frame_interval
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

# Function to generate video captions from frames
def generate_video_caption(video_path, frame_interval=180):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, frame_interval)
    
    if not frames:
        return ""  # No frames to process if extraction failed
    
    captions = []
    
    for frame in tqdm(frames, desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    
    return " ".join(captions)  # Concatenate all captions into one string

# Function to extract audio and convert to text using speech recognition
def extract_audio_text(video_path):
    # Use ffmpeg to extract audio
    audio_path = "temp_audio.wav"
    os.system(f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 2 {audio_path}")
    
    # Use speech recognition to transcribe the audio
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)  # read the entire audio file
        try:
            transcript = recognizer.recognize_google(audio)
            os.remove(audio_path)  # Delete the temp audio file
            return transcript
        except sr.UnknownValueError:
            return "Audio could not be transcribed"
        except sr.RequestError as e:
            return f"Could not request results from Google Speech Recognition service; {e}"

# Main function to process a video and save captions/audio transcript to CSV
def process_video(video_path, output_csv="video_captions.csv"):
    caption = generate_video_caption(video_path)
    audio_transcript = extract_audio_text(video_path)
    
    data = {
        "video_name": os.path.basename(video_path),
        "caption": caption,
        "audio_transcript": audio_transcript
    }

    df = pd.DataFrame([data])
    
    # Append to CSV file
    df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

# Function to process all videos in a folder
def process_videos_in_folder(folder_path, output_csv="video_captionss.csv"):
    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    for video_file in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(folder_path, video_file)
        process_video(video_path, output_csv)

# Example usage
folder_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/.cartoonmovies7_7298084692916555041.mp4"  # Replace with the folder containing your video files
process_videos_in_folder(folder_path)  # This will process all videos in the folder and save captions and audio transcripts to CSV


In [ ]:
import cv2
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm
import subprocess

# Function to extract exactly 5 frames from the video
def extract_frames(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    # Ensure that we have at least 'num_frames' frames to extract
    if total_frames < num_frames:
        print(f"Warning: Video has fewer frames than the requested {num_frames}. Extracting all available frames.")
        num_frames = total_frames
    
    # Calculate the frame indices to extract (evenly spaced frames)
    frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]
    
    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)  # Set the video capture to the correct frame
        ret, frame = cap.read()
        if not ret:
            break
        
        # Convert the frame from BGR to RGB (for PIL compatibility)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame_rgb)
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

# Function to generate video captions from frames
def generate_video_caption(video_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, num_frames=5)
    
    if not frames:
        return ""  # No frames to process if extraction failed
    
    captions = []
    
    for frame in tqdm(frames, desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    
    return " ".join(captions)  # Concatenate all captions into one string

# Function to extract audio and convert to text using speech recognition
def extract_audio_text(video_path):
    audio_path = "temp_audio.wav"  # The path where audio will be saved
    
    # Remove the existing audio file if it exists to ensure it will be overwritten
    if os.path.exists(audio_path):
        os.remove(audio_path)
        print(f"Removed existing audio file: {audio_path}")
    
    # Use ffmpeg to extract audio
    ffmpeg_command = f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 2 {audio_path}"
    print(f"Running command: {ffmpeg_command}")
    
    try:
        # Run the ffmpeg command to extract audio
        subprocess.run(ffmpeg_command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error in ffmpeg command: {e}")
        return "Audio extraction failed"
    
    # Check if the audio file was created successfully
    if not os.path.exists(audio_path):
        print(f"Error: {audio_path} not created.")
        return "Audio extraction failed"
    
    # Check if the audio file is empty
    if os.path.getsize(audio_path) == 0:
        print(f"Error: {audio_path} is empty.")
        return "Audio extraction failed"
    
    # Use speech recognition to transcribe the audio
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)  # Read the entire audio file
        try:
            print("Transcribing audio...")
            transcript = recognizer.recognize_google(audio)
            os.remove(audio_path)  # Delete the temp audio file after processing
            return transcript
        except sr.UnknownValueError:
            print("Speech Recognition could not understand the audio.")
            return "Audio could not be transcribed"
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return f"Could not request results from Google Speech Recognition service; {e}"
            
# Main function to process a video and save captions/audio transcript to CSV
def process_video(video_path, output_csv="video_captionss3.csv"):
    caption = generate_video_caption(video_path)
    audio_transcript = extract_audio_text(video_path)
    
    data = {
        "video_name": os.path.basename(video_path),
        "caption": caption,
        "audio_transcript": audio_transcript
    }

    df = pd.DataFrame([data])
    
    # Append to CSV file
    df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

# Function to process all videos in a folder
def process_videos_in_folder(folder_path, output_csv="video_captionss3.csv"):
    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    for video_file in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(folder_path, video_file)
        process_video(video_path, output_csv)

# Example usage
folder_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/"  # Replace with the folder containing your video files
process_videos_in_folder(folder_path)  # This will process all videos in the folder and save captions and audio transcripts to CSV


In [ ]:
import cv2
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm

# Function to extract frames from video
def extract_frames(video_path, frame_interval=180):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Add frame if the index is a multiple of frame_interval
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

# Function to generate video captions from frames
def generate_video_caption(video_path, frame_interval=180):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, frame_interval)
    
    if not frames:
        return ""  # No frames to process if extraction failed
    
    captions = []
    
    for frame in tqdm(frames, desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    
    return " ".join(captions)  # Concatenate all captions into one string

# Function to extract audio and convert to text using speech recognition
def extract_audio_text(video_path):
    audio_path = "temp_audio.wav"  # The path where audio will be saved
    
    # Remove the existing audio file if it exists to ensure it will be overwritten
    if os.path.exists(audio_path):
        os.remove(audio_path)
        print(f"Removed existing audio file: {audio_path}")
    
    # Use ffmpeg to extract audio
    ffmpeg_command = f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 2 {audio_path}"
    print(f"Running command: {ffmpeg_command}")
    
    try:
        # Run the ffmpeg command to extract audio
        subprocess.run(ffmpeg_command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error in ffmpeg command: {e}")
        return "Audio extraction failed"
    
    # Check if the audio file was created successfully
    if not os.path.exists(audio_path):
        print(f"Error: {audio_path} not created.")
        return "Audio extraction failed"
    
    # Check if the audio file is empty
    if os.path.getsize(audio_path) == 0:
        print(f"Error: {audio_path} is empty.")
        return "Audio extraction failed"
    
    # Use speech recognition to transcribe the audio
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)  # Read the entire audio file
        try:
            print("Transcribing audio...")
            transcript = recognizer.recognize_google(audio)
            os.remove(audio_path)  # Delete the temp audio file after processing
            return transcript
        except sr.UnknownValueError:
            print("Speech Recognition could not understand the audio.")
            return "Audio could not be transcribed"
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return f"Could not request results from Google Speech Recognition service; {e}"

# Main function to process a video and save captions/audio transcript to CSV
def process_video(video_path, output_csv="video_captions.csv"):
    caption = generate_video_caption(video_path)
    audio_transcript = extract_audio_text(video_path)
    
    data = {
        "video_name": os.path.basename(video_path),
        "caption": caption,
        "audio_transcript": audio_transcript
    }

    df = pd.DataFrame([data])
    
    # Append to CSV file
    df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

# Function to process all videos in a folder
def process_videos_in_folder(folder_path, output_csv="video_captions.csv"):
    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    for video_file in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(folder_path, video_file)
        process_video(video_path, output_csv)

# Example usage
folder_path = "/path/to/your/folder"  # Replace with the folder containing your video files
process_videos_in_folder(folder_path)  # This will process all videos in the folder and save captions and audio transcripts to CSV


In [ ]:
import cv2
import os
import pandas as pd
import speech_recognition as sr
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm
import torch
import os
import speech_recognition as sr
import subprocess



# Function to extract frames from video
def extract_frames(video_path, frame_interval=200):
    cap = cv2.VideoCapture(video_path)
    
    # Check if video was loaded correctly
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    frames = []
    frame_num = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        if frame_num % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frames.append(frame_rgb)
        
        frame_num += 1
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames

# Function to generate video captions from frames
def generate_video_caption(video_path, frame_interval=200):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)

    frames = extract_frames(video_path, frame_interval)
    if not frames:
        return ""  # No frames to process if extraction failed
    
    captions = []
    
    for frame in tqdm(frames, desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt").to(device)
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)
    
    return " ".join(captions)  # Concatenate all captions into one string

# Function to extract audio and convert to text using speech recognition
def extract_audio_text(video_path):
    audio_path = "temp_audio.wav"  # The path where audio will be saved
    
    # Remove the existing audio file if it exists to ensure it will be overwritten
    if os.path.exists(audio_path):
        os.remove(audio_path)
        print(f"Removed existing audio file: {audio_path}")
    
    # Use ffmpeg to extract audio
    ffmpeg_command = f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 2 {audio_path}"
    print(f"Running command: {ffmpeg_command}")
    
    try:
        # Run the ffmpeg command to extract audio
        subprocess.run(ffmpeg_command, shell=True, check=True)
    except subprocess.CalledProcessError as e:
        print(f"Error in ffmpeg command: {e}")
        return "Audio extraction failed"
    
    # Check if the audio file was created successfully
    if not os.path.exists(audio_path):
        print(f"Error: {audio_path} not created.")
        return "Audio extraction failed"
    
    # Check if the audio file is empty
    if os.path.getsize(audio_path) == 0:
        print(f"Error: {audio_path} is empty.")
        return "Audio extraction failed"
    
    # Use speech recognition to transcribe the audio
    recognizer = sr.Recognizer()
    with sr.AudioFile(audio_path) as source:
        audio = recognizer.record(source)  # Read the entire audio file
        try:
            print("Transcribing audio...")
            transcript = recognizer.recognize_google(audio)
            os.remove(audio_path)  # Delete the temp audio file after processing
            return transcript
        except sr.UnknownValueError:
            print("Speech Recognition could not understand the audio.")
            return "Audio could not be transcribed"
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return f"Could not request results from Google Speech Recognition service; {e}"

# Main function to process the video and save captions/audio transcript to CSV
def process_video(video_path, output_csv="video_captions1.csv"):
    caption = generate_video_caption(video_path)
    audio_transcript = extract_audio_text(video_path)
    
    data = {
        "video_name": os.path.basename(video_path),
        "caption": caption,
        "audio_transcript": audio_transcript
    }

    df = pd.DataFrame([data])
    
    # Append to CSV file
    df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

# Example usage
video_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/gocnhaclive_7127954090977398018.mp4"  # Replace with your video file path
process_video(video_path)  # This will process the video and save captions and audio text to CSV


In [ ]:
#this is the oneeee
import cv2
import os
import pandas as pd
import speech_recognition as sr
from pydub import AudioSegment
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm
import subprocess  # Add this import to handle the ffmpeg command

def extract_frames(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: Unable to open video file at {video_path}")
        return []
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Total frames in video: {total_frames}")
    
    if total_frames < num_frames:
        print(f"Warning: Video has fewer frames than the requested {num_frames}.")
        num_frames = total_frames
    
    frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]
    print(f"Frame indices to extract: {frame_indices}")

    frames = []
    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            print(f"Failed to read frame at index {idx}")
            break
        frames.append(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    
    cap.release()
    print(f"Extracted {len(frames)} frames.")
    return frames


# Function to generate video captions from frames
def generate_video_caption(video_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, num_frames=5)
    
    if not frames:
        return ""  # No frames to process if extraction failed
    
    captions = []
    
    for frame in tqdm(frames, desc="Processing Frames"):
        print(f"Processing frame with shape: {frame.shape}")  # Check the shape of the frame
        if frame is None:
            print("Frame is None, skipping.")
            continue
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)

    
    return " ".join(captions)  # Concatenate all captions into one string

# # Function to extract audio and convert to text using speech recognition
# def extract_audio_text(video_path):
#     audio_path = "temp_audio.wav"  # The path where audio will be saved
    
#     # Remove the existing audio file if it exists to ensure it will be overwritten
#     if os.path.exists(audio_path):
#         os.remove(audio_path)
#         print(f"Removed existing audio file: {audio_path}")
    
#     # Use ffmpeg to extract audio
#     ffmpeg_command = f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 44100 -ac 2 {audio_path}"
#     print(f"Running command: {ffmpeg_command}")
    
#     try:
#         # Run the ffmpeg command to extract audio
#         subprocess.run(ffmpeg_command, shell=True, check=True)
#     except subprocess.CalledProcessError as e:
#         print(f"Error in ffmpeg command: {e}")
#         return "Audio extraction failed"
    
#     # Check if the audio file was created successfully
#     if not os.path.exists(audio_path):
#         print(f"Error: {audio_path} not created.")
#         return "Audio extraction failed"
    
#     # Check if the audio file is empty
#     if os.path.getsize(audio_path) == 0:
#         print(f"Error: {audio_path} is empty.")
#         return "Audio extraction failed"
    
#     # Use speech recognition to transcribe the audio
#     recognizer = sr.Recognizer()
#     with sr.AudioFile(audio_path) as source:
#         audio = recognizer.record(source)  # Read the entire audio file
#         try:
#             print("Transcribing audio...")
#             transcript = recognizer.recognize_google(audio)
#             os.remove(audio_path)  # Delete the temp audio file after processing
#             return transcript
#         except sr.UnknownValueError:
#             print("Speech Recognition could not understand the audio.")
#             return "Audio could not be transcribed"
#         except sr.RequestError as e:
#             print(f"Could not request results from Google Speech Recognition service; {e}")
#             return f"Could not request results from Google Speech Recognition service; {e}"

# Main function to process a video and save captions/audio transcript to CSV
def process_video(video_path, output_csv="video_captions6.csv"):
    caption = generate_video_caption(video_path)
    # audio_transcript = extract_audio_text(video_path)
    
    data = {
        "video_name": os.path.basename(video_path),
        "caption": caption,
        # "audio_transcript": audio_transcript
    }

    df = pd.DataFrame([data])
    
    # Append to CSV file
    df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)

# Function to process all videos in a folder
def process_videos_in_folder(folder_path, output_csv="video_captions6.csv"):
    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
    
    for video_file in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(folder_path, video_file)
        # Get file size in bytes and convert to megabytes
        file_size_mb = os.path.getsize(video_path) / (1024 * 1024)
        
        # Print file name and file size
        print(f"File Name: {video_file}, File Size: {file_size_mb:.2f} MB")
        if video_path == "/kaggle/input/tikharm-dataset/Dataset/train/Safe/gocnhaclive_7127954090977398018.mp4":
            continue
        process_video(video_path, output_csv)

# Example usage
folder_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/"  # Replace with the folder containing your video files
process_videos_in_folder(folder_path)  # This will process all videos in the folder and save captions and audio transcripts to CSV


Processing Videos:   0%|          | 0/698 [00:00<?, ?it/s]

File Name: sam.shan.shops_7360833729805257990.mp4, File Size: 7.03 MB


/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1617: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be deprecated in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Total frames in video: 1086
Frame indices to extract: [0, 217, 434, 651, 868]
Extracted 5 frames.



Processing Frames:   0%|          | 0/5 [00:00<?, ?it/s]/opt/conda/lib/python3.10/site-packages/transformers/generation/utils.py:1220: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(

Processing Videos:   0%|          | 1/698 [00:26<5:10:52, 26.76s/it]

...


Processing Videos: 100%|█████████▉| 696/698 [3:40:14<00:38, 19.04s/it]

File Name: maythangtuw_7229686566879300869.mp4, File Size: 5.94 MB
Total frames in video: 1050
Frame indices to extract: [0, 210, 420, 630, 840]
Extracted 5 frames.



Processing Videos: 100%|█████████▉| 697/698 [3:40:32<00:18, 18.81s/it]

File Name: dicungtuito_7273869341215460615.mp4, File Size: 9.28 MB
Total frames in video: 1837
Frame indices to extract: [0, 367, 734, 1102, 1469]
Extracted 5 frames.



Processing Videos: 100%|██████████| 698/698 [3:40:51<00:00, 18.98s/it]


In [ ]:
import cv2
import os
import pandas as pd
from transformers import BlipProcessor, BlipForConditionalGeneration
from PIL import Image
from tqdm import tqdm
import subprocess
import logging

# Setup logging for debugging
logging.basicConfig(level=logging.INFO, format="%(asctime)s [%(levelname)s] %(message)s")

# Function to extract exactly 5 frames from the video
def extract_frames(video_path, num_frames=5):
    cap = cv2.VideoCapture(video_path)
    frames = []

    # Check if video was loaded correctly
    if not cap.isOpened():
        logging.error(f"Error: Unable to open video file at {video_path}")
        return frames

    # Get total number of frames in the video
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    if total_frames < num_frames:
        logging.warning(f"Video has fewer frames ({total_frames}) than requested ({num_frames}). Extracting all available frames.")
        num_frames = total_frames

    # Calculate frame indices
    frame_indices = [int(i * total_frames / num_frames) for i in range(num_frames)]

    for idx in frame_indices:
        cap.set(cv2.CAP_PROP_POS_FRAMES, idx)
        ret, frame = cap.read()
        if not ret:
            logging.error(f"Error: Could not read frame {idx} from {video_path}")
            continue

        # Convert frame to RGB
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frames.append(frame_rgb)

        # Safety pause to prevent infinite hangs
        cv2.waitKey(1)

    cap.release()
    logging.info(f"Extracted {len(frames)} frames from {video_path}")
    return frames

# Function to generate video captions
def generate_video_caption(video_path):
    processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-base")
    model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-base")
    
    frames = extract_frames(video_path, num_frames=5)
    if not frames:
        return ""  # Return empty if no frames were extracted
    
    captions = []
    for frame in tqdm(frames, desc="Processing Frames"):
        image = Image.fromarray(frame)
        inputs = processor(images=image, return_tensors="pt")
        out = model.generate(**inputs)
        caption = processor.decode(out[0], skip_special_tokens=True)
        captions.append(caption)

    return " ".join(captions)

# Main function to process video
def process_video(video_path, output_csv="video_captions6.csv"):
    try:
        logging.info(f"Processing video: {video_path}")
        caption = generate_video_caption(video_path)
        data = {
            "video_name": os.path.basename(video_path),
            "caption": caption,
        }
        df = pd.DataFrame([data])
        df.to_csv(output_csv, mode='a', header=not os.path.exists(output_csv), index=False)
    except Exception as e:
        logging.error(f"Error processing video {video_path}: {e}")

# Function to process all videos in a folder
def process_videos_in_folder(folder_path, output_csv="video_captions6.csv"):
    video_files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mov'))]
    for video_file in tqdm(video_files, desc="Processing Videos"):
        video_path = os.path.join(folder_path, video_file)
        process_video(video_path, output_csv)

# Example usage
folder_path = "/kaggle/input/tikharm-dataset/Dataset/train/Safe/"  # Update this path
process_videos_in_folder(folder_path)


In [ ]:
import cv2

def watch_video(video_path):
    # Open the video file
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print(f"Error: Cannot open video file {video_path}")
        return

    while True:
        ret, frame = cap.read()
        if not ret:  # End of video
            break
        
        cv2.imshow("Video", frame)
        
        # Break on 'q' key press
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

# Example usage
watch_video("/kaggle/input/tikharm-dataset/Dataset/train/Safe/gocnhaclive_7127954090977398018.mp4")  # Replace with your video path
